In [2]:
import pandas as pd
import numpy as np

In [3]:
Default=pd.read_csv('Default.csv',)
Default.head()

,Unnamed: 0,default,student,balance,income
0,1,No,No,729.526495,44361.625074
1,2,No,Yes,817.180407,12106.134700
2,3,No,No,1073.549164,31767.138947
3,4,No,No,529.250605,35704.493935
4,5,No,No,785.655883,38463.495879


In [4]:
import statsmodels.formula.api as smf

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [6]:
le=LabelEncoder()
Default['student01'] = le.fit_transform(Default['student'])

In [7]:
Default['default01'] = (Default.default == 'Yes').astype(int)
Default.head()

,Unnamed: 0,default,student,balance,income,student01,default01
0,1,No,No,729.526495,44361.625074,0,0
1,2,No,Yes,817.180407,12106.134700,1,0
2,3,No,No,1073.549164,31767.138947,0,0
3,4,No,No,529.250605,35704.493935,0,0
4,5,No,No,785.655883,38463.495879,0,0


In [8]:
Default_train=Default.head(int(len(Default)*0.8)).reset_index(drop=True)
Default_test=Default.tail(len(Default) - int(len(Default)*0.8)).reset_index(drop=True)
Default_test.head()

,Unnamed: 0,default,student,balance,income,student01,default01
0,8001,No,No,304.146598,23866.935858,0,0
1,8002,No,No,353.631131,40686.760148,0,0
2,8003,No,No,1269.420335,39010.148187,0,0
3,8004,No,Yes,397.542488,22710.865740,1,0
4,8005,No,No,607.166271,49547.465816,0,0


In [9]:
k=7
model = KNeighborsClassifier(n_neighbors=k)
model.fit(Default_train[['balance','income','student01']],Default_train['default'])

KNeighborsClassifier(n_neighbors=7)

In [10]:
Default_train['pred_label_knn']=model.predict(Default_train[['balance','income','student01']])
Default_test['pred_label_knn']=model.predict(Default_test[['balance','income','student01']])

In [11]:
accuracy_rate=sum(Default_test['pred_label_knn']==Default_test['default'])/len(Default_test)
ER=1-accuracy_rate
ER

0.031000000000000028

In [12]:
Default_train['pred_prob_knn']=model.predict_proba(Default_train[['balance','income','student01']])[:,1]
Default_test['pred_prob_knn']=model.predict_proba(Default_test[['balance','income','student01']])[:,1]

In [13]:
from sklearn.metrics import *

In [14]:
roc_auc_score(y_true=Default_test['default'],y_score = Default_test['pred_prob_knn'])

0.821806641906865

# 逻辑回归

In [15]:
import statsmodels.formula.api as s

In [16]:
Default.default == 'Yes'

0       False
1       False
2       False
3       False
4       False
        ...  
9995    False
9996    False
9997    False
9998    False
9999    False
Name: default, Length: 10000, dtype: bool

In [17]:
Default['default01'] = (Default.default == 'Yes').astype(int)
Default.head()

,Unnamed: 0,default,student,balance,income,student01,default01
0,1,No,No,729.526495,44361.625074,0,0
1,2,No,Yes,817.180407,12106.134700,1,0
2,3,No,No,1073.549164,31767.138947,0,0
3,4,No,No,529.250605,35704.493935,0,0
4,5,No,No,785.655883,38463.495879,0,0


In [18]:
results = smf.logit('default01 ~ balance + student + income', data=Default).fit()
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.078577
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:              default01   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9996
Method:                           MLE   Df Model:                            3
Date:                Sat, 04 Dec 2021   Pseudo R-squ.:                  0.4619
Time:                        11:03:40   Log-Likelihood:                -785.77
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                3.257e-292
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        -10.8690      0.492    -22.079      0.000     -11.834      -9.904
student[T.Yes]  

In [19]:
Default_train=Default.head(int(len(Default)*0.8)).reset_index(drop=True)
Default_test=Default.tail(len(Default) - int(len(Default)*0.8)).reset_index(drop=True)
Default_test.head()

,Unnamed: 0,default,student,balance,income,student01,default01
0,8001,No,No,304.146598,23866.935858,0,0
1,8002,No,No,353.631131,40686.760148,0,0
2,8003,No,No,1269.420335,39010.148187,0,0
3,8004,No,Yes,397.542488,22710.865740,1,0
4,8005,No,No,607.166271,49547.465816,0,0


In [20]:
results = smf.logit('default01 ~ balance + student + income', data=Default_train).fit()
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.077231
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:              default01   No. Observations:                 8000
Model:                          Logit   Df Residuals:                     7996
Method:                           MLE   Df Model:                            3
Date:                Sat, 04 Dec 2021   Pseudo R-squ.:                  0.4705
Time:                        11:03:40   Log-Likelihood:                -617.85
converged:                       True   LL-Null:                       -1166.9
Covariance Type:            nonrobust   LLR p-value:                9.276e-238
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        -11.1808      0.564    -19.837      0.000     -12.285     -10.076
student[T.Yes]  

In [21]:
results_mul = smf.logit('default01 ~ student+balance+income', data=Default).fit()
print(results_mul.summary())

Optimization terminated successfully.
         Current function value: 0.078577
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:              default01   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9996
Method:                           MLE   Df Model:                            3
Date:                Sat, 04 Dec 2021   Pseudo R-squ.:                  0.4619
Time:                        11:03:41   Log-Likelihood:                -785.77
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                3.257e-292
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        -10.8690      0.492    -22.079      0.000     -11.834      -9.904
student[T.Yes]  

In [22]:
default_prob_pred=results_mul.predict(Default)
default_pred=(default_prob_pred>0.5).astype(int)

In [23]:
accuracy_rate=sum(default_pred==Default['default01'])/len(Default)
accuracy_rate
ER2=1-accuracy_rate
ER2

0.026800000000000046

In [24]:
roc_auc_score(y_true=Default['default01'],y_score = default_pred)

0.6555887634815947